<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [62]:
try:
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import multiprocessing
    from gensim.models import Word2Vec
except Exception as e:
    print("Error al importar:", e)

### Datos
Utilizaremos como dataset el libro "Martin Fierro".

In [63]:
import pandas as pd
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# Leer el libro archivo línea por línea contenido en el archivo martin_fierro.txt
with open('martin_fierro.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

# Convertir a DataFrame
df = pd.DataFrame(lines, columns=['historia'])

# Tokenizar cada línea
sentence_tokens = []
for _, row in df.iterrows():
    sentence_tokens.append(text_to_word_sequence(row['historia']))

# Mostrar algunos tokens
for tokens in sentence_tokens[:10]:
    print(tokens)


['el', 'gaucho', 'martín', 'fierro']
['josé', 'hernández']
[]
['1']
[]
[]
['texto', 'núm', '4187']
['título', 'el', 'gaucho', 'martín', 'fierro']
['autor', 'josé', 'hernández']
['etiquetas', 'poesía']


In [64]:
with open('martin_fierro.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()


df = pd.DataFrame(lines, columns=['historia'])
df.head()


,historia
0,El Gaucho Martín Fierro
1,José Hernández
2,
3,1
4,


In [65]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2449


### 1 - Preprocesamiento

In [66]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []

for _, row in df.iterrows():
    sentence_tokens.append(text_to_word_sequence(row.iloc[0]))


In [67]:
# Demos un vistazo
sentence_tokens[:2]

[['el', 'gaucho', 'martín', 'fierro'], ['josé', 'hernández']]

### 2 - Crear los vectores (word2vec)

In [68]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [69]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=3,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [70]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [71]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2449


In [72]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 557


### 3 - Entrenar embeddings

In [73]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 89008.3671875
Loss after epoch 1: 39834.140625
Loss after epoch 2: 30790.8984375
Loss after epoch 3: 28836.046875
Loss after epoch 4: 28303.34375
Loss after epoch 5: 28390.4375
Loss after epoch 6: 27790.828125
Loss after epoch 7: 27644.5
Loss after epoch 8: 28414.3125
Loss after epoch 9: 28205.78125
Loss after epoch 10: 27970.625
Loss after epoch 11: 27716.875
Loss after epoch 12: 27839.125
Loss after epoch 13: 28404.09375
Loss after epoch 14: 27645.71875
Loss after epoch 15: 27883.28125
Loss after epoch 16: 28609.4375
Loss after epoch 17: 28383.1875
Loss after epoch 18: 29201.25
Loss after epoch 19: 27680.0


(94709, 225780)

### 4 - Ensayar

In [74]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["el"], topn=10)

[('desgraciao', 0.999029815196991),
 ('hace', 0.9989508390426636),
 ('eso', 0.9989194273948669),
 ('juerte', 0.9989127516746521),
 ('ese', 0.9989070892333984),
 ('padecer', 0.9989000558853149),
 ('gato', 0.9988833665847778),
 ('aguanta', 0.9988819360733032),
 ('parece', 0.9988793134689331),
 ('arriba', 0.998876690864563)]

In [75]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["cantar"], topn=10)

[('vez', -0.9896597266197205),
 ('tal', -0.9900179505348206),
 ('he', -0.99434494972229),
 ('visto', -0.9965164065361023),
 ('o', -0.9972394108772278),
 ('esa', -0.9972704648971558),
 ('no', -0.9977584481239319),
 ('qué', -0.9978582262992859),
 ('mesmo', -0.9978677034378052),
 ('hay', -0.9979145526885986)]

In [76]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["ande"], topn=10)

[('digo', 0.9994636178016663),
 ('dentra', 0.9994292855262756),
 ('redomón', 0.9994131326675415),
 ('lomo', 0.9994101524353027),
 ('duro', 0.9994089007377625),
 ('güelta', 0.9994075894355774),
 ('pingo', 0.9994034171104431),
 ('negro', 0.9993978142738342),
 ('bagual', 0.9993978142738342),
 ('moro', 0.9993952512741089)]

In [77]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["padre"], topn=5)

[('andan', 0.9995988607406616),
 ('lomo', 0.9995643496513367),
 ('tres', 0.9995617866516113),
 ('atrás', 0.9995545148849487),
 ('sargento', 0.9995543360710144)]

In [78]:
# Ensayar con una palabra que no está en el vocabulario: esta palabra da error porque no esta en el dioccionario
w2v_model.wv.most_similar(negative=["bandido"])

KeyError: "Key 'bandido' not present in vocabulary"

In [79]:
# el método `get_vector` permite obtener los vectores:
vector_cantar = w2v_model.wv.get_vector("cantar")
print(vector_cantar)

[-0.00050781  0.2085369  -0.06500391  0.05736004 -0.00975889 -0.17992401
  0.17077117  0.3129725  -0.00903545 -0.04222796  0.0275842  -0.10434504
 -0.08883115  0.05350181 -0.04573907 -0.10932864  0.08587694 -0.01221502
  0.09318512 -0.01396328 -0.03025087 -0.04882079  0.10858749  0.03946175
  0.08637434 -0.00161521 -0.140606    0.03199051 -0.16930047 -0.18132226
  0.09581241 -0.16078734  0.0455488   0.02948486 -0.03675834  0.00842973
  0.12354623 -0.18974361 -0.02900615 -0.00854821 -0.02140403 -0.0380908
  0.06241261 -0.03704563 -0.0281008   0.08867034  0.01622634 -0.08564964
 -0.01358147  0.09397678  0.00062217  0.03433939 -0.11237222  0.09785821
 -0.05706544  0.05991004  0.12803021  0.02283803  0.03350443 -0.05834982
 -0.04172467 -0.02563581 -0.01392855  0.02155487 -0.03369389  0.06768615
  0.08464661 -0.01224829 -0.06556129 -0.00744201  0.03633333 -0.02451801
  0.11885854 -0.0550122   0.08493529  0.07457112 -0.10137311  0.02181086
 -0.16268167  0.10109476 -0.11183537 -0.06784387 -0.

In [80]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_cantar)

[('cantar', 0.9999999403953552),
 ('güelta', 0.9993950128555298),
 ('puse', 0.9993795156478882),
 ('camino', 0.9993779063224792),
 ('hago', 0.9993709921836853),
 ('servir', 0.9993592500686646),
 ('gritos', 0.9993585348129272),
 ('meses', 0.9993576407432556),
 ('destino', 0.9993570446968079),
 ('trabajar', 0.9993553161621094)]

### 5 - Visualizar agrupación de vectores

In [81]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [82]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [83]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [88]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

In [44]:
w2v_model.wv.most_similar(positive=["gaucho", "autoridá"], negative=["libre"])

[('aguanta', 0.998572826385498),
 ('menos', 0.998545229434967),
 ('hoyo', 0.9985441565513611),
 ('desgraciao', 0.9985345005989075),
 ('comendante', 0.9985234141349792),
 ('güena', 0.998515248298645),
 ('anda', 0.9985134601593018),
 ('sol', 0.9985074400901794),
 ('pata', 0.9984993934631348),
 ('rigor', 0.9984962344169617)]

#### Interpretación:
1. aguanta, rigor, desgraciao
Estas palabras reflejan la condición de opresión y sufrimiento. El gaucho sometido a la autoridad, privado de su libertad, es quien "aguanta" el "rigor", y se convierte en un "desgraciao".

In [85]:
w2v_model.wv.most_similar(positive=["desgracias", "pobres"], negative=["frontera"])

[('jogón', 0.9991311430931091),
 ('inorancia', 0.9991233348846436),
 ('criollo', 0.9991176724433899),
 ('estrellas', 0.9991169571876526),
 ('dicen', 0.9991165399551392),
 ('alguno', 0.9991087317466736),
 ('juir', 0.9991050362586975),
 ('prendas', 0.9990985989570618),
 ('pasaba', 0.9990982413291931),
 ('siquiera', 0.9990974068641663)]

#### Interpretación:
2. jogón, inorancia, criollo
Estas palabras capturan elementos del entorno y del tipo social caracterizado por la pobreza y las penurias.
jogón (fogón rústico) alude a la vida doméstica precaria.
inorancia (forma arcaica de ignorancia) señala la falta de educación como consecuencia de la marginalización.
criollo puede reforzar el perfil del habitante rural y humilde.

In [87]:
w2v_model.wv.most_similar(positive = ["china", "penas"], negative=["alegría"])

[('boca', 0.9992053508758545),
 ('negra', 0.9991989731788635),
 ('lengua', 0.9991660714149475),
 ('esas', 0.9991601705551147),
 ('falte', 0.9991515874862671),
 ('tarde', 0.999151349067688),
 ('cantor', 0.999147891998291),
 ('tres', 0.999134361743927),
 ('frontera', 0.9991329908370972),
 ('punta', 0.9991316199302673)]

#### Interpretación:
3. desgracias, pobres, frontera
Estas palabras aparecen reiteradamente en contextos de miseria, exclusión y represión. El modelo parece reforzar que el sufrimiento de la “china” se vincula al entorno opresivo y marginal del universo gauchesco.

### Conclusión

El modelo, a pesar de haber sido entrenado con un corpus literario de estilo arcaico y poético, logra aprender relaciones semánticas coherentes con la narrativa del poema. No requiere de términos técnicos ni explícitos para entender:

- Relación entre roles sociales (gaucho, china, coronel).

- Condiciones de vida (pobreza, dolor, trabajo).

- Conflictos morales y políticos (libertad vs. autoridad).

Además, utiliza elementos simbólicos y culturales propios del texto, como jogón, frontera, criollo, o autoridá, para representar conceptos complejos sin necesidad de un lenguaje moderno o técnico.

El modelo es sensible al contexto cultural y estilístico del corpus.

La calidad del resultado depende de la frecuencia y el uso contextual de las palabras.

Las analogías permiten explorar la "visión de mundo" que el modelo ha construido, y son herramientas útiles para análisis semántico, literario y cultural.

Este ejercicio demuestra que incluso con un corpus literario específico, es posible entrenar modelos de lenguaje que capturen relaciones profundas y útiles para el análisis del discurso, la literatura o incluso para tareas de humanidades digitales.